In [1]:
import pandas as pd


In [2]:
data = pd.read_csv('drug_consumption.csv')

,ID,Age,Gender,Education,Country,Ethnicity,Nscore,Escore,Oscore,Ascore,...,Ecstasy,Heroin,Ketamine,Legalh,LSD,Meth,Mushrooms,Nicotine,Semer,VSA
0,1,0.49788,0.48246,-0.05921,0.96082,0.12600,0.31287,-0.57545,-0.58331,-0.91699,...,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL2,CL0,CL0
1,2,-0.07854,-0.48246,1.98437,0.96082,-0.31685,-0.67825,1.93886,1.43533,0.76096,...,CL4,CL0,CL2,CL0,CL2,CL3,CL0,CL4,CL0,CL0
2,3,0.49788,-0.48246,-0.05921,0.96082,-0.31685,-0.46725,0.80523,-0.84732,-1.62090,...,CL0,CL0,CL0,CL0,CL0,CL0,CL1,CL0,CL0,CL0
3,4,-0.95197,0.48246,1.16365,0.96082,-0.31685,-0.14882,-0.80615,-0.01928,0.59042,...,CL0,CL0,CL2,CL0,CL0,CL0,CL0,CL2,CL0,CL0
4,5,0.49788,0.48246,1.98437,0.96082,-0.31685,0.73545,-1.63340,-0.45174,-0.30172,...,CL1,CL0,CL0,CL1,CL0,CL0,CL2,CL2,CL0,CL0


In [3]:
## Age recoding
age_col = {
          -0.95197: '18-24',
          -0.07854: '25 - 34',
          0.49788: '35 - 44',
          1.09449: '45 - 54',
          1.82213: '55 - 64',
          2.59171: '65+'
          }
data['Age'] = data['Age'].replace(age_col)

In [4]:
## Gender recoding
gender_col = {
            0.48246: 'Female',
            -0.48246: 'Male'
            }
data['Gender'] = data['Gender'].replace(gender_col)

In [5]:
## Education recoding
education_col = {
            -2.43591: 'Left School Before 16 years',
            -1.73790: 'Left School at 16 years',
            -1.43719: 'Left School at 17 years',
            -1.22751: 'Left School at 18 years',
            -0.61113: 'Some College,No Certificate Or Degree',
            -0.05921: 'Professional Certificate/ Diploma',
            0.45468: 'University Degree',
            1.16365: 'Masters Degree',
            1.98437: 'Doctorate Degree',
            }
data['Education'] = data['Education'].replace(education_col)

In [6]:
## Country recoding
country_col = {
            -0.09765: 'Australia',
            0.24923: 'Canada',
            -0.46841: 'New Zealand',
            -0.28519: 'Other',
            0.21128: 'Republic of Ireland',
            0.96082: 'UK',
            -0.57009: 'USA'
            }
data['Country'] = data['Country'].replace(country_col)

In [7]:
## Ethnicity recoding
ethnicity_col = {
            -0.50212: 'Asian',
            -1.10702: 'Black',
            1.90725: 'Mixed-Black/Asian',
            0.12600: 'Mixed-White/Asian',
            -0.22166: 'Mixed-White/Black',
            0.11440: 'Other',
            -0.31685: 'White'
            }
data['Ethnicity'] = data['Ethnicity'].replace(ethnicity_col)

In [8]:
## Usage recoding
usage_col = {
    'CL0': 'Never Used',
    'CL1': 'Used over a Decade Ago',
    'CL2': 'Used in Last Decade',
    'CL3': 'Used in Last Year',
    'CL4': 'Used in Last Month',
    'CL5': 'Used in Last Week',
    'CL6': 'Used in Last Day',
    }
data['Alcohol'] = data['Alcohol'].replace(usage_col)
data['Amphet'] = data['Amphet'].replace(usage_col)
data['Amyl'] = data['Amyl'].replace(usage_col)
data['Benzos'] = data['Benzos'].replace(usage_col)
data['Caff'] = data['Caff'].replace(usage_col)
data['Cannabis'] = data['Cannabis'].replace(usage_col)
data['Choc'] = data['Choc'].replace(usage_col)
data['Coke'] = data['Coke'].replace(usage_col)
data['Crack'] = data['Crack'].replace(usage_col)
data['Ecstasy'] = data['Ecstasy'].replace(usage_col)
data['Heroin'] = data['Heroin'].replace(usage_col)
data['Ketamine'] = data['Ketamine'].replace(usage_col)
data['Legalh'] = data['Legalh'].replace(usage_col)
data['LSD'] = data['LSD'].replace(usage_col)
data['Meth'] = data['Meth'].replace(usage_col)
data['Mushrooms'] = data['Mushrooms'].replace(usage_col)
data['Nicotine'] = data['Nicotine'].replace(usage_col)
data['Semer'] = data['Semer'].replace(usage_col)
data['VSA'] = data['VSA'].replace(usage_col)

In [9]:
data.head()

,ID,Age,Gender,Education,Country,Ethnicity,Nscore,Escore,Oscore,Ascore,...,Ecstasy,Heroin,Ketamine,Legalh,LSD,Meth,Mushrooms,Nicotine,Semer,VSA
0,1,35 - 44,Female,Professional Certificate/ Diploma,UK,Mixed-White/Asian,0.31287,-0.57545,-0.58331,-0.91699,...,Never Used,Never Used,Never Used,Never Used,Never Used,Never Used,Never Used,Used in Last Decade,Never Used,Never Used
1,2,25 - 34,Male,Doctorate Degree,UK,White,-0.67825,1.93886,1.43533,0.76096,...,Used in Last Month,Never Used,Used in Last Decade,Never Used,Used in Last Decade,Used in Last Year,Never Used,Used in Last Month,Never Used,Never Used
2,3,35 - 44,Male,Professional Certificate/ Diploma,UK,White,-0.46725,0.80523,-0.84732,-1.62090,...,Never Used,Never Used,Never Used,Never Used,Never Used,Never Used,Used over a Decade Ago,Never Used,Never Used,Never Used
3,4,18-24,Female,Masters Degree,UK,White,-0.14882,-0.80615,-0.01928,0.59042,...,Never Used,Never Used,Used in Last Decade,Never Used,Never Used,Never Used,Never Used,Used in Last Decade,Never Used,Never Used
4,5,35 - 44,Female,Doctorate Degree,UK,White,0.73545,-1.63340,-0.45174,-0.30172,...,Used over a Decade Ago,Never Used,Never Used,Used over a Decade Ago,Never Used,Never Used,Used in Last Decade,Used in Last Decade,Never Used,Never Used


In [10]:
##make a classement of the drugs by the number of users
data['Alcohol'].value_counts()
data['Amphet'].value_counts()
data['Amyl'].value_counts()
data['Benzos'].value_counts()
data['Caff'].value_counts()
data['Cannabis'].value_counts()
data['Choc'].value_counts()
data['Coke'].value_counts()
data['Crack'].value_counts()
data['Ecstasy'].value_counts()
data['Heroin'].value_counts()
data['Ketamine'].value_counts()
data['Legalh'].value_counts()
data['LSD'].value_counts()
data['Meth'].value_counts()
data['Mushrooms'].value_counts()
data['Nicotine'].value_counts()
data['Semer'].value_counts()
data['VSA'].value_counts()



VSA
Never Used                1455
Used over a Decade Ago     200
Used in Last Decade        135
Used in Last Year           61
Used in Last Week           14
Used in Last Month          13
Used in Last Day             7
Name: count, dtype: int64

In [31]:
drugs = ['Alcohol', 'Amphet', 'Amyl', 'Benzos', 'Caff', 'Cannabis', 'Choc', 'Coke', 'Crack', 'Ecstasy', 'Heroin', 'Ketamine', 'Legalh', 'LSD', 'Meth', 'Mushrooms', 'Nicotine', 'Semer', 'VSA']

# Create an empty DataFrame
drug_counts = pd.DataFrame()

# Rename the index column to 'Frequency'
drug_counts = drug_counts.rename_axis('Frequency')

# Loop through each drug
for drug in drugs:
    # Get the count of each category for this drug
    counts = data[drug].value_counts()
    # Add these counts as a new column in the DataFrame
    drug_counts[drug] = counts

# Display the DataFrame
print(drug_counts)

                        Alcohol  Amphet  Amyl  Benzos  Caff  Cannabis  Choc  \
Frequency                                                                     
Used in Last Week           759      61    14      84   273       185   683   
Used in Last Day            505     102     3      95  1385       463   807   
Used in Last Month          287      75    24     120   106       140   296   
Used in Last Year           198     198    92     236    60       211    54   
Used in Last Decade          68     243   237     234    24       266    10   
Used over a Decade Ago       34     230   210     116    10       207     3   
Never Used                   34     976  1305    1000    27       413    32   

                        Coke  Crack  Ecstasy  Heroin  Ketamine  Legalh   LSD  \
Frequency                                                                      
Used in Last Week         41      9       63      16        33      64    56   
Used in Last Day          19      2       21    

In [17]:
drugs = ['Alcohol', 'Amphet', 'Amyl', 'Benzos', 'Caff', 'Cannabis', 'Choc', 'Coke', 'Crack', 'Ecstasy', 'Heroin', 'Ketamine', 'Legalh', 'LSD', 'Meth', 'Mushrooms', 'Nicotine', 'Semer', 'VSA']

# Create an empty DataFrame
drug_counts = pd.DataFrame()

# Loop through each drug
for drug in drugs:
    # Get the count of each category for this drug
    counts = data[drug].value_counts()
    # Add these counts as a new column in the DataFrame
    drug_counts[drug] = counts

# Reset the index of the DataFrame
drug_counts.reset_index(inplace=True)

# Rename the index column to 'Frequency'
drug_counts.rename(columns={'index': 'Frequency'}, inplace=True)

# Display the DataFrame
print(drug_counts)

ValueError: cannot insert Alcohol, already exists